## 문자 문제

유니코드 표준은 문자의 단위 원소와 특정 바이트 표현을 명확히 구분한다.

문자를 표현하는 실제 바이트는 사용하는 인코딩에 따라 달라진다.
인코딩은 코드 포인트를 바이트 시퀀스로 변환하는 알고리즘이다.

코드 포인트를 바이트로 변환하는 것을 **인코딩**, 바이트를 코드 포인트로 변환하는 것을 **디코딩**이라고 한다.

In [ ]:
s = 'café'
len(s)

b = s.encode('utf8')
print(b)
len(b)
b.decode('utf8')

bytes 시퀀스는 알아보기 어려운 기계 메모리 덤프로, 유니코드 str은 '사람'이 읽을 수 있는 텍스트로 생각하면 된다. 그러면 bytes 시퀀스를 사람이 읽을 수 있는
텍스트로 **디코딩(해독)** 하고, str을 저장하거나 전송하기 위해 bytes로 **인코딩(암호화)** 한다는 말이 이해될 것이다.

## 바이트에 대한 기본 지식
bytes 형은 파이썬 3에서 소개된 불변형이고, bytearray는 파이썬 2.6에 추가된 가변형이다
bytes와 bytearray에 들어 있는 각 항목은 0에서 255 사이의 정수로, 파이썬 2의 str에 들어있는 문자 하나로 구성된 문자열과는 다르다.
그러나 이진 시퀀스를 슬라이싱하면 언제나 도일한 자료형의 이진 시퀀스가 만들어지며, 슬라이스 길이가 1일 때도 마찬가지다.

In [1]:
cafe = bytes('café', encoding='utf8')
print(cafe)

print(cafe[0])
print(cafe[:1])

cafe_arr = bytearray(cafe)
print(cafe_arr)
print(cafe_arr[-1:])

b'caf\xc3\xa9'
99
b'c'
bytearray(b'caf\xc3\xa9')
bytearray(b'\xa9')


## 인코딩/디코딩 문제 이해하기
UnicodeError라는 범용 예외가 있지만, 거의 항상 UnicodeEncodeError(str을 이진 시퀀스로 변환할 때)나 UnicodeDecodeError(이진 시퀀스를 str로 읽어 들일 때) 같은 구체적인 예외가 발생한다.
파이썬 모듈을 로딩할 때 소스코드가 예기치 않은 방식으로 인코딩되어 있으면 SyntaxError가 발생하기도 한다.

## BOM(Byte order mark): 유용한 깨진 문자
UTF-16에는 리틀엔디언을 명시하는 UTF-16LE와 빅엔디언을 명시하는 UTF-16BE 변형이 있다.
이들 변형 인코딩을 사용하면 BOM을 생성하지 않는다.
존재하는 경우 BOM은 UTF 16코덱에 의해 걸러지므로, 텍스트로 변환한 후에는 앞에 추가된 WIDTH NO-BREAK SPACE 문자 없이 파일의 실제 내용만 가져온다.
표준에 의하면 UTF-16파일에 BOM이 없다면 이 파일은 UTF-16BE(빅엔디언)로 코딩되어 있다고 가정해야 한다. 그렇지만 인텔 x86 아키텍처가 리틀엔디언이므로 실제로는 BOM이 없지만 UTF-16LE(리틀엔디언)로 인코딩된 파일도 많다.

## 제대로 비교하기 위해 유니코드 정규화하기
유니코드에는 결합 문자가 있기 떄문에 문자열 비교가 간단하지 않다. 앞 문자에 연결되는 발음 구별 기호는 인쇄할 때 앞 문자와 하나로 결합되어 출력된다.

에를 들어 cafe` 라는 단어는 네 개나 다섯 개의 코드 포인트를 이용해서 두 가지 방식으로 표현할 수 있자만 결과는 동일하게 나타난다.
유니코드에서는 두 개의 시퀀스를 규범적으로 동일하다고 하며, 애플리케이션은 이 두 시퀀스를 동일하게 처리해야 한다.
그렇지만 파이썬은 서로 다른 두 개의 코드 포인트 시퀀스를 보고, 이 둘이 서로 동일하지 않다고 판단한다.

이 문제를 해결하려면 unicodedata.normalize() 함수가 제공하는 유니코드 정규화를 이용해야 한다.
이 함수의 첫 번째 인수는 'NFC', 'NFD', 'NFKC', 'NFKD'중 하나여야 한다.

## 케이스 폴딩
본질적으로 케이스 폴딩은 모든 텍스트를 소문자로 변환하는 연산이며, 약간의 변환을 동반한다. 케이스 폴딩은 파이썬 3.3에 추가된 str.casefold() 메서드를 이용해서 수행한다.

latin1

In [ ]:
# UnicodeEncodeError 처리하기